# Prosperity trading algorithm

## Set up

Import all nessesary dependencies

In [1]:
from datamodel import Listing, Trade, OrderDepth, UserId, TradingState, Order
from typing import List
import string

An example `TradingState` object `state` is created for testing locally. It is initialized with the following parameters:

In [2]:
timestamp = 1000

listings = {
	"AMETHYSTS": Listing(
		symbol="AMETHYSTS", 
		product="AMETHYSTS", 
		denomination="SEASHELLS"
	),
	"STARFRUIT": Listing(
		symbol="STARFRUIT", 
		product="STARFRUIT",
		denomination="SEASHELLS"
	),
}

order_depths = {
	"AMETHYSTS": OrderDepth(
		buy_orders={10: 7, 9: 5},
		sell_orders={11: -4, 12: -8}
	),
	"STARFRUIT": OrderDepth(
		buy_orders={142: 3, 141: 5},
		sell_orders={144: -5, 145: -8}
	),	
}

own_trades = {
	"AMETHYSTS": [],
	"STARFRUIT": []
}

market_trades = {
	"AMETHYSTS": [
		Trade(
			symbol="AMETHYSTS",
			price=11,
			quantity=4,
			buyer="",
			seller="",
			timestamp=900
		)
	],
	"STARFRUIT": []
}

position = {
	"AMETHYSTS": 3,
	"STARFRUIT": -5
}

observations = {}
traderData = ""

state = TradingState(
	traderData,
	timestamp,
    listings,
	order_depths,
	own_trades,
	market_trades,
	position,
	observations
)

### Set current position limit

In [3]:
position_limit = {'AMETHYSTS': 20, 'STARFRUIT': 20}

## Trader class

An example trader class

In [4]:
class Trader:
    
    def run(self, state: TradingState):
        # Only method required. It takes all buy and sell orders for all symbols as an input, and outputs a list of orders to be sent
        print("traderData: " + state.traderData)
        print("Observations: " + str(state.observations))
        
        result = {}
        for product in state.order_depths:
            order_depth: OrderDepth = state.order_depths[product]
            orders: List[Order] = []
            acceptable_price = 10;  # Participant should calculate this value
            
            # print('Product: ' + product)
            # print("Acceptable price : " + str(acceptable_price))
            # print("Buy Order depth : " + str(len(order_depth.buy_orders)) + ", Sell order depth : " + str(len(order_depth.sell_orders)))
    
            if len(order_depth.sell_orders) != 0:
                best_ask, best_ask_amount = list(order_depth.sell_orders.items())[0]
                if int(best_ask) < acceptable_price:
                    print("BUY", str(-best_ask_amount) + "x", best_ask)
                    orders.append(Order(product, best_ask, -best_ask_amount))
    
            if len(order_depth.buy_orders) != 0:
                best_bid, best_bid_amount = list(order_depth.buy_orders.items())[0]
                if int(best_bid) > acceptable_price:
                    print("SELL", str(best_bid_amount) + "x", best_bid)
                    orders.append(Order(product, best_bid, -best_bid_amount))
            
            result[product] = orders
    
    
        traderData = "SAMPLE" # String value holding Trader state data required. It will be delivered as TradingState.traderData on next execution.
        
        conversions = 1
        return result, conversions, traderData

Test trader class locally

In [5]:
trader = Trader()
result, conversions, traderData = trader.run(state)

traderData: 
Observations: {}
Product: AMETHYSTS
Acceptable price : 10
Buy Order depth : 2, Sell order depth : 2
Product: STARFRUIT
Acceptable price : 10
Buy Order depth : 2, Sell order depth : 2
SELL 3x 142


In [6]:
result

{'AMETHYSTS': [], 'STARFRUIT': [(STARFRUIT, 142, -3)]}

In [12]:
conversions

1

In [13]:
traderData

'SAMPLE'

## Load returned data from the trading simulation

In [6]:
import pandas as pd

data = pd.read_csv('6c4eadcc-7119-4003-8dfb-97a54eba15b6.csv',sep=';')
data.head(-20)

,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,-2,0,AMETHYSTS,10002,1,9996.0,2.0,9995.0,29.0,10004,2,10005.0,29.0,NaN,NaN,10003.0,0.000000
1,-2,0,STARFRUIT,5002,1,4997.0,31.0,NaN,NaN,5003,31,NaN,NaN,NaN,NaN,5002.5,0.000000
2,-2,100,AMETHYSTS,9996,2,9995.0,22.0,NaN,NaN,10004,2,10005.0,22.0,NaN,NaN,10000.0,2.000000
3,-2,100,STARFRUIT,4997,24,NaN,NaN,NaN,NaN,5003,2,5004.0,22.0,NaN,NaN,5000.0,1.925293
4,-2,200,STARFRUIT,4997,2,4996.0,20.0,NaN,NaN,5003,20,NaN,NaN,NaN,NaN,5000.0,-52.304688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,-2,198700,AMETHYSTS,9996,2,9995.0,20.0,NaN,NaN,10004,2,10005.0,20.0,NaN,NaN,10000.0,1386.000000
3976,-2,198800,AMETHYSTS,9996,1,9995.0,27.0,NaN,NaN,10004,1,10005.0,27.0,NaN,NaN,10000.0,1386.000000
3977,-2,198800,STARFRUIT,4948,28,NaN,NaN,NaN,NaN,4949,1,4955.0,28.0,NaN,NaN,4948.5,914.148438
3978,-2,198900,STARFRUIT,4949,8,4948.0,27.0,NaN,NaN,4950,5,4955.0,27.0,NaN,NaN,4949.5,915.289062


## Load returned trading log from the trading simulation
This is to identify potential issues with the trading algorithm, e.g. exceeding the position limit